<center>
<img src="../../img/ml_theme.png">
# Дополнительное профессиональное <br> образование НИУ ВШЭ
#### Программа "Практический анализ данных и машинное обучение"
<img src="../../img/faculty_logo.jpg" height="240" width="240">
## Автор материала: Лисицын Сергей
</center>
Материал распространяется на условиях лицензии <a href="https://opensource.org/licenses/MS-RL">Ms-RL</a>. Можно использовать в любых целях, кроме коммерческих, но с обязательным упоминанием автора материала.

# <center>Занятие 9. Разреженные данные, онлайн-обучение</center>

## <center>Часть 2. Vowpal Wabbit</center>

Vowpal Wabbit (VW) является одной из наиболее широкоиспользуемых библиотек в индустрии. Её отличает высокая скорость работы и поддержка большого количества различных режимов обучения. Особый интерес для больших и высокоразмерных данных представляет онлайн-обучение - самая сильная сторона библиотеки. 


Основным интерфейсом для работы с VW является shell

In [1]:
import sklearn.datasets
import sklearn.cross_validation
import sklearn.metrics
import re

import matplotlib.pyplot as plt
%matplotlib inline

plt.style.use('ggplot')

/Users/lisitsyn/anaconda3/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/Users/lisitsyn/anaconda3/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


Для того, чтобы изучить возможные режимы работы vw воспользуемся командой ! для запуска:

In [2]:
!vw --help

Num weight bits = 18
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = 
num sources = 1


VW options:
  --random_seed arg                     seed random number generator
  --ring_size arg                       size of example ring

Update options:
  -l [ --learning_rate ] arg            Set learning rate
  --power_t arg                         t power value
  --decay_learning_rate arg             Set Decay factor for learning_rate 
                                        between passes
  --initial_t arg                       initial t value
  --feature_mask arg                    Use existing regressor to determine 
                                        which parameters may be updated.  If no
                                        initial_regressor given, also used for 
                                        initial weights.

Weight options:
  -i [ --initial_regressor ] arg        Initial regressor(s)
  --initial_weight arg                  Set all 

Vowpal Wabbit считывает данные из файла или стандартного ввода (stdin) в формате, который имеет следующий вид:

`[Label] [Importance] [Tag]|Namespace Features |Namespace Features ... |Namespace Features`

`Namespace=String[:Value]`

`Features=(String[:Value] )*`

где [] обозначает необязательные элементы, а (...)\* означает повтор неопределенное число раз. 

- **Label** является числом, "правильным" ответом. В случае классификации обычно принимает значение 1/-1, а в случае регрессии некоторое вещественное число
- **Importance** является числом и отвечает за вес примера при обучении. Это позволяет бороться с проблемой несбалансированных данных, изученной нами ранее
- **Tag** является некоторой строкой без пробелов и отвечает за некоторое "название" примера, которое сохраняется при предсказании ответа. Для того, чтобы отделить Tag от Importance лучше начинать Tag с символа '.
- **Namespace** служит для создания отдельных пространств признаков. В аргументах Namespace именуются по первой букве, это нужно учитывать при выборе их названий
- **Features** являются непосредственно признаками объекта внутри **Namespace**. Признаки по умолчанию имеют вес 1.0, но его можно переопределить, к примеру feature:0.1. 


К примеру, под такой формат подходит следующая строка:

```
1 1.0 |Subject WHAT car is this |Organization University of Maryland:0.5 College Park
```


чтобы убедиться в этом, запустим vw с этим обучающим примером:

In [3]:
! echo '1 1.0 |Subject WHAT car is this |Organization University of Maryland:0.5 College Park' | vw

For more information use: vw --help
Num weight bits = 18
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = 
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
1.000000 1.000000            1            1.0   1.0000   0.0000       10

finished run
number of examples per pass = 1
passes used = 1
weighted example sum = 1.000000
weighted label sum = 1.000000
average loss = 1.000000
best constant = 1.000000
best constant's loss = 0.000000
total feature number = 10


VW является прекрасным инструментом для работы с текстовыми данными. Убедимся в этом с помощью выборки 20newsgroups, содержащей письма из 20 различных тематических рассылок:

In [4]:
newsgroups = sklearn.datasets.fetch_20newsgroups('../../data/')

Рассмотрим первый текстовый документ этой коллекции:

In [5]:
text = newsgroups['data'][0]
target = newsgroups['target_names'][newsgroups['target'][0]]

print('-----')
print(target)
print('-----')
print(text.strip())
print('----')

SyntaxError: Missing parentheses in call to 'print' (<ipython-input-5-5032ddc5c67e>, line 4)

Выполним простейшую обработку текста: избавимся от регистра и найдем все слова длиннее трех символов:

In [ ]:
def vowpalize(document, label=None):
    return str(label or '') + ' |text ' + ' '.join(re.findall('\w{3,}', text.lower())) + '\n'

vowpalize(text, 1 if target == 'rec.autos' else -1)

Разобьем выборку на обучающую и тестовую и запишем в файл преобразованные таким образом документы. Будем считать документ положительным, если он относится к рассылке 'rec.autos'. Так мы построим модель, отличающую письма про автомобили от остальных: 

In [ ]:
all_documents = newsgroups['data']
all_targets = [1 if newsgroups['target_names'][target] == 'rec.autos' else -1 for target in newsgroups['target']]

train_documents, test_documents, train_labels, test_labels = \
    sklearn.cross_validation.train_test_split(all_documents, all_targets)
    
with open('../../output/20news_train.vw', 'w') as vw_train_data:
    for text, target in zip(train_documents, train_labels):
        vw_train_data.write(vowpalize(text, target))
with open('../../output/20news_test.vw', 'w') as vw_test_data:
    for text in test_documents:
        vw_test_data.write(vowpalize(text))

Запустим vw на сформированном файле. Мы решаем задачу классификации, поэтому зададим функцию потерь в значение hinge. В следующей части занятия мы изучим поддерживаемые vw функции потерь. Построенную модель мы сохраним в соответствующий файл 20news_model.vw:

In [ ]:
!vw -d ../../output/20news_train.vw --loss_function hinge -f ../../output/20news_model.vw

Модель обучена. VW выводит достаточно много полезной информации по ходу обучения. Обратите внимание, что average loss снижался по ходу выполнения итераций. Для вычисления функции потерь VW использует еще не просмотренные примеры, поэтому как правило эта оценка является корректной. Применим обученную модель на тестовой выборке, сохраняя предсказания в файл с помощью опции -p: 

In [ ]:
!vw -i ../../output/20news_model.vw -t -d ../../output/20news_test.vw -p ../../output/20news_test_predictions.txt

Загрузим полученные предсказания, вычислим AUC и отобразим ROC-кривую:

In [ ]:
test_predicted_labels = map(float, open('../../output/20news_test_predictions.txt').readlines())

auc = sklearn.metrics.roc_auc_score(test_labels, test_predicted_labels)
roc_curve = sklearn.metrics.roc_curve(test_labels, test_predicted_labels)

plt.plot(roc_curve[0], roc_curve[1]);
plt.plot([0,1], [0,1])
plt.xlabel('FPR'); plt.ylabel('TPR'); plt.title('test AUC = %f' % (auc)); plt.axis([-0.05,1.05,-0.05,1.05]);

Полученное значения AUC говорит о высоком качестве классификации.

### Ссылки

- https://github.com/JohnLangford/vowpal_wabbit/wiki